# **Cleaning Zelenskyy's channel**

Messages are either in Ukrainian, English or both (divided by '––'). Messages in Ukrainian are often followed by another message with the English translation. Hence, a way to clean the data set is to match Ukrainian messages with their English equivalent (more below). Message types:

1.   Ukrainian only: are often followed by an English equivalent
2.   English only: majorly translations of Ukrainian messages + a small fraction are not equivalents

Of interest are enagement metrics by Ukrainians, i.e. focusing on Ukrainian messages.

There are three `dfs`:

1.  `df`: raw data
2.  `df_clean`: cleaned df
3.  `df_clean_trans`: cleaned df with translations
---

File includes the following steps:

1.   **Basic Cleaning**
  *   Removing rows without message (media only) and messages form other channels (focus on messages by the president only)
  *   Clean emoijs, i.e. splitting reactions in separate columns
  *   Remove rows before the beginning of the war


2.   **Match messages**: the idea is to either translate/or match messages
  *   Translation: would be the easiest approach, simply translates Ukrainian messages to English + takes the English part of messages in both languages
  *   Match equivalents: can match English equivalent message to Ukrainian. E.g. by using cosine similarity on embeddings or BOW





In [ ]:
%%capture

# Load packages
!pip install deep_translator
!pip install spacy_langdetect
!pip install sentence-transformers

# Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time
import ast
import re

# Texts and languages
from sklearn.preprocessing import normalize
from deep_translator import GoogleTranslator

import nltk
from nltk.tokenize import sent_tokenize

from spacy_langdetect import LanguageDetector
import spacy
from spacy.language import Language
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Embeddings
# Mini LM sentence embeddsing
from sentence_transformers import SentenceTransformer
emb_model_mini = SentenceTransformer("all-MiniLM-L6-v2")

# Previously I use USE
# import tensorflow_hub as hub
# emb_model_use = hub.load( "https://tfhub.dev/google/universal-sentence-encoder/4")
# print ("module %s loaded" % module_url)

In [ ]:
# Load colab packages
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


__Dataframes__

There are three `dfs`:

1.    `df` is the raw data to be cleaned resulting in
2.    `df_clean` resulting from basic cleaning
3.    `df_trans_val` adds translations to the cleaned `df`
4.    `df_clean_trans` shows the translation validity on messages containing both languages (more below)

In [ ]:
# Load raw data
df = pd.read_csv('/content/drive/My Drive/01_LSE/2_Courses/ST/Dissertation/3_Ukraine_emotive_rethoric/Data/Raw/df_zelen_raw.csv')

In [ ]:
df.head(10)
df.shape

(6995, 12)

## Basic cleaning

Includes:



*   dropping rows without messages, dropping rows from other channels. Rows without messages are media files. If a single message is send with four images, the data fout contains rows, one with the text and image and three rows with the image only.
*   Clean the emoij colunmn
*   Remove rows before the war began



In [ ]:
# Two functions to clean rows and process emoijs column
def drop_rows_vars(df, name):
    ''''''
    df_out = df.copy()
    print(name, 'rows without texts', df_out.shape[0] - df_out['message'].count())
    df_out = df_out.dropna(subset=['message'])

    # Drop messages that are from other channels
    print(name, 'messages from other channels', df_out['fwd_from'].count())
    df_out = df_out[df_out['fwd_from'].isna()]

    # Drop the forward from columns
    df_out.drop(['noforwards', 'from_id', 'fwd_from', 'button_count', 'Unnamed: 0'], axis=1, inplace=True)
    return df_out

# Emoij column is not used at the end
def clean_emoijs(df):
    ''''''

    # Make the column a list
    df_out = df.copy()
    df_out['reactions'] = df_out['reactions'].apply(ast.literal_eval)

    # Get unique emoijs
    unique_emoijs = [r[0] for el in df_out['reactions'] for r in el]
    unique_emoijs = set(unique_emoijs)

    # For each, create a column
    for column_name in unique_emoijs:
        df_out[column_name] = 0  # Assigning None as the default value

    # Get emoij counts
    for index, row in df_out.iterrows():
        # Iterate over the list
        for el in row['reactions']:
            df_out.loc[index, el[0]] = el[1]

    # Total emoijs
    df_out['Total_emoijs'] = df_out[unique_emoijs].sum(axis=1)
    return df_out

In [ ]:
def cleaning(df, name):

    out = drop_rows_vars(df, name)
    out = clean_emoijs(out)

    return out

In [ ]:
# ------------------------------
# Apply cleaning
df_clean = cleaning(df, 'V_Zelenskiy_official')

# ------------------------------
# Get only messages after the beginning of the war
df_clean['date'] = pd.to_datetime(df_clean['date'])  # Convert the column to datetime if it's not already
rows_tot = df_clean.shape[0]
df_clean = df_clean[df_clean['date'] > pd.to_datetime('2022-02-24').tz_localize('UTC')]
print('Rows before war', rows_tot - df_clean.shape[0])
print('New total rows', df_clean.shape[0])

# Reset index
df_clean = df_clean.reset_index(drop=True)


V_Zelenskiy_official rows without texts 3502
V_Zelenskiy_official messages from other channels 1
Rows before war 508
New total rows 2984


<ipython-input-5-6e95d65f867d>:38: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_out['Total_emoijs'] = df_out[unique_emoijs].sum(axis=1)


In [ ]:
# Overview data
df_clean.head(10)

,id,date,message,views,forwards,reactions,mime_type,👍,🙏,❤,😢,Total_emoijs
0,7045,2023-07-19 09:29:04+00:00,На ранковій селекторній нараді заслухав інформ...,21820.0,92.0,"[(❤, 515), (👍, 139), (🙏, 71), (😢, 20)]",text,139,71,515,20,745
1,7044,2023-07-18 20:55:39+00:00,I held a meeting of the Staff. The main issues...,107222.0,36.0,"[(❤, 3146), (👍, 340), (🙏, 206), (😢, 44)]",video/mp4,340,206,3146,44,3736
2,7043,2023-07-18 19:41:43+00:00,Провів засідання Ставки. Основні питання – це ...,1371997.0,565.0,"[(❤, 3702), (👍, 370), (🙏, 210), (😢, 51)]",video/mp4,370,210,3702,51,4333
3,7042,2023-07-18 17:31:58+00:00,Зустрівся з президентом Японського агентства м...,327240.0,229.0,"[(❤, 3879), (👍, 480), (🙏, 192), (😢, 34)]",video/mp4,480,192,3879,34,4585
4,7041,2023-07-18 15:42:48+00:00,Війна – це зло. \nВійна – це трагедія.\nВійна ...,233477.0,294.0,"[(❤, 3689), (👍, 489), (🙏, 275), (😢, 59)]",video/mp4,489,275,3689,59,4512
5,7040,2023-07-18 10:39:31+00:00,Питання номер один на сьогоднішній Ставці – мо...,442594.0,255.0,"[(👍, 3322), (❤, 1159), (🙏, 296), (😢, 37)]",text,3322,296,1159,37,4814
6,7030,2023-07-18 09:23:45+00:00,Український прапор хоч і пробитий кулями – але...,375414.0,255.0,"[(❤, 4453), (👍, 404), (🙏, 300), (😢, 31)]",image/jpeg,404,300,4453,31,5188
7,7029,2023-07-17 20:32:39+00:00,Говорив з Генеральним секретарем ООН Антоніу Г...,405124.0,229.0,"[(👍, 3816), (❤, 1339), (🙏, 355), (😢, 52)]",text,3816,355,1339,52,5562
8,7028,2023-07-17 20:12:49+00:00,I have sent official letters to President of T...,151007.0,86.0,"[(❤, 3574), (👍, 412), (🙏, 206), (😢, 36)]",video/mp4,412,206,3574,36,4228
9,7027,2023-07-17 19:32:33+00:00,Я направив офіційні листи Президенту Туреччини...,1517840.0,967.0,"[(❤, 5060), (👍, 700), (🙏, 260), (😢, 40)]",video/mp4,700,260,5060,40,6060


## Match messages

There are two options:

1. Match messages as a majority of Ukrainian messages have an english equivalent

2. Translate Ukrainian messages

Comparing to texts is done using cosine similarity on BOW feature matrixes and on word embeddings. See _Translation validation_ below. For embeddings I currently use the universal sentence encoder which suitable for short paragraphs (see https://tfhub.dev/google/universal-sentence-encoder/4).

DO: may be too high-dimensional, hence reducing dimensionality using UMAP could improve results (that's the algorithm used by Top2Vec). Or just use the allMini embedding!


### Processing to prepare matching


Notes:

*   See functions below of what is done here
*   Translation is done once as is takes to much time to run everytime, and saved in a separate data file




In [ ]:
# ------------------------------------
# Run this once only

# For the language detection
def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)

In [ ]:
# FUNCTIONS

# ------------------------------------
# Language detection
# ------------------------------------

def detect_lan(text):
    '''Takes text and returns language and score.'''
    doc = nlp(text)
    return doc._.language

# ------------------------------------
# Language similarity
# ------------------------------------

vectorizer = TfidfVectorizer()

def bow_similarity(text1: str, text2: str) -> float:
    '''Takes two texts and computes the cosine similairty
    based on tf-idf weighted bag of words.'''

    vectors = vectorizer.fit_transform( [text1, text2])
    matrix = cosine_similarity(vectors)
    return matrix[0][1]

# ------------------------------------
# Embeddings
# ------------------------------------

def get_embedding(text: str):
  # For USE
  #em = np.array(emb_model_mini([text])[0])

# For allMini
  em = emb_model_mini.encode(text)
  return em

def embedding_similarity(em1, em2):
  return 1 - cosine(em1, em2)

# Split texts (specific to the channel; messages containing both languages are divided by --- or --)
def split_texts(text: str):
  '''Takes the first occurence of either ukraine or english'''
  if '____' in text:
      out = text.split('____')
      langs = [detect_lan(t).get('language') for t in out]

      if 'en' not in langs and 'uk' in langs:
        return None, out[langs.index('uk')]
      if 'en' in langs and 'uk' not in langs:
        return out[langs.index('en')], None
      return out[langs.index('en')], out[langs.index('uk')]
  if '——' in text:
      out = text.split('——')
      langs = [detect_lan(t).get('language') for t in out]

      if 'en' not in langs and 'uk' in langs:
        return None, out[langs.index('uk')]
      if 'en' in langs and 'uk' not in langs:
        return out[langs.index('en')], None
      return out[langs.index('en')], out[langs.index('uk')]

  return text


# ------------------------------------
# Translation: Google API
# ------------------------------------
# translator = GoogleTranslator(source='uk', target='en')
# def translate(text):
#     '''Takes text and translates it from Ukrainian in to English.'''

#     attempts=0
#     while attempts < 3:
#         try:
#           # If text to long
#           if len(text) > 5000:
#             sentences = sent_tokenize(text)
#             text_out = []
#             for s in sentences:
#               text_out.append(translator.translate(s))
#               if text_out:
#                 return ' '.join(text_out)
#               else:
#                 return None
#           else:
#             text_out = translator.translate(text) # Execute the code
#             return text_out
#         except Exception as e:
#             time.sleep(3)
#             attempts += 1
#             if attempts == 3:
#                 print("Error encountered on the third attempt.")
#                 return None
#             else:
#                 print(f"Attempt {attempts} failed. Retrying...")
#                 continue

In [ ]:
# TEST FUNCTIONS
text1 = 'Is this sentence the same as the second?'
text2 = 'Is this sentence the same as the first?'

print('Language detection, ', detect_lan('Which language is this?'))
print('BOW similarity, ', bow_similarity(text1,
                 text2))
print('Embedding detection, ', embedding_similarity(get_embedding(text1), get_embedding(text2)))
print('Splitting text, ', split_texts('This is the one text —— Станом на цей час на території Херсонської та Миколаївської областей евакуйовано вже понад три тисячі людей. —— This is another language'))
#print('Translation ', translate("Станом на цей час на території Херсонської та Миколаївської областей евакуйовано вже понад три тисячі людей."))

Language detection,  {'language': 'en', 'score': 0.9999960275062558}
BOW similarity,  0.8200207392821179
Embedding detection,  0.9174867868423462
Splitting text,  ('This is the one text ', ' Станом на цей час на території Херсонської та Миколаївської областей евакуйовано вже понад три тисячі людей. ')


In [ ]:
# ------------------------------------
# APPLY TO DATA
# Used function for a clean global space

# 1. Split messages that have both languages in to two columns
df_clean['message_both'] = df_clean['message'].apply(lambda x: '____' in x or '——' in x).astype(int)
df_clean['message_split_en'], df_clean['message_split_uk'] = None, None

def split_langs():
  for index, row in df_clean.iterrows():
    if row['message_both'] == 1:
      en, uk = split_texts(row['message'])
      df_clean['message_split_en'].iloc[index], df_clean['message_split_uk'].iloc[index] = en, uk
split_langs()

# 2. Get lan of columns with unary language
df_clean['message_lan'], df_clean['message_score'] = None, None

def message_lan():

  for index, row in df_clean.iterrows():

    # Main message
    if row['message_both'] == 0:
      lan = detect_lan(row['message'])
      df_clean['message_lan'].iloc[index] = lan.get('language')
      df_clean['message_score'].iloc[index] = lan.get('score')

message_lan()

<ipython-input-12-a2a17a459478>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['message_split_en'].iloc[index], df_clean['message_split_uk'].iloc[index] = en, uk
<ipython-input-12-a2a17a459478>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['message_lan'].iloc[index] = lan.get('language')
<ipython-input-12-a2a17a459478>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['message_score'].iloc[index] = lan

In [ ]:
df_clean.head(10)

,id,date,message,views,forwards,reactions,mime_type,👍,🙏,❤,😢,Total_emoijs,message_both,message_split_en,message_split_uk,message_lan,message_score
0,7045,2023-07-19 09:29:04+00:00,На ранковій селекторній нараді заслухав інформ...,21820.0,92.0,"[(❤, 515), (👍, 139), (🙏, 71), (😢, 20)]",text,139,71,515,20,745,0,None,None,uk,0.999997
1,7044,2023-07-18 20:55:39+00:00,I held a meeting of the Staff. The main issues...,107222.0,36.0,"[(❤, 3146), (👍, 340), (🙏, 206), (😢, 44)]",video/mp4,340,206,3146,44,3736,0,None,None,en,0.999997
2,7043,2023-07-18 19:41:43+00:00,Провів засідання Ставки. Основні питання – це ...,1371997.0,565.0,"[(❤, 3702), (👍, 370), (🙏, 210), (😢, 51)]",video/mp4,370,210,3702,51,4333,0,None,None,uk,0.999997
3,7042,2023-07-18 17:31:58+00:00,Зустрівся з президентом Японського агентства м...,327240.0,229.0,"[(❤, 3879), (👍, 480), (🙏, 192), (😢, 34)]",video/mp4,480,192,3879,34,4585,0,None,None,uk,0.999997
4,7041,2023-07-18 15:42:48+00:00,Війна – це зло. \nВійна – це трагедія.\nВійна ...,233477.0,294.0,"[(❤, 3689), (👍, 489), (🙏, 275), (😢, 59)]",video/mp4,489,275,3689,59,4512,1,\n\nWar is evil. \nWar is tragedy.\nWar is a t...,Війна – це зло. \nВійна – це трагедія.\nВійна ...,None,None
5,7040,2023-07-18 10:39:31+00:00,Питання номер один на сьогоднішній Ставці – мо...,442594.0,255.0,"[(👍, 3322), (❤, 1159), (🙏, 296), (😢, 37)]",text,3322,296,1159,37,4814,0,None,None,uk,0.999999
6,7030,2023-07-18 09:23:45+00:00,Український прапор хоч і пробитий кулями – але...,375414.0,255.0,"[(❤, 4453), (👍, 404), (🙏, 300), (😢, 31)]",image/jpeg,404,300,4453,31,5188,1,"\n\nThe Ukrainian flag, although riddled with ...",Український прапор хоч і пробитий кулями – але...,None,None
7,7029,2023-07-17 20:32:39+00:00,Говорив з Генеральним секретарем ООН Антоніу Г...,405124.0,229.0,"[(👍, 3816), (❤, 1339), (🙏, 355), (😢, 52)]",text,3816,355,1339,52,5562,0,None,None,uk,0.999997
8,7028,2023-07-17 20:12:49+00:00,I have sent official letters to President of T...,151007.0,86.0,"[(❤, 3574), (👍, 412), (🙏, 206), (😢, 36)]",video/mp4,412,206,3574,36,4228,0,None,None,en,0.999995
9,7027,2023-07-17 19:32:33+00:00,Я направив офіційні листи Президенту Туреччини...,1517840.0,967.0,"[(❤, 5060), (👍, 700), (🙏, 260), (😢, 40)]",video/mp4,700,260,5060,40,6060,0,None,None,uk,0.999999


### Create final message column (previously message_final section)




In [ ]:
def final_message():
  #df_out = df_clean.copy()
  df_clean['en_original'] = 0
  df_clean['uk_trans'] = 0
  df_clean['message_final'] = None

  for index, row in df_clean.iterrows():

    if row['message_both'] == 1 and row['message_split_en']:
        df_clean['message_final'].iloc[index] = row['message_split_en']
        df_clean['en_original'].iloc[index] = 1
    if row['message_both'] == 1 and row['message_split_en'] is None:
        df_clean['uk_trans'].iloc[index] = 1
    if row['message_both'] == 0 and row['message_lan'] == 'uk':
      df_clean['uk_trans'].iloc[index] = 1

final_message()

<ipython-input-14-437a3419d628>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['uk_trans'].iloc[index] = 1
<ipython-input-14-437a3419d628>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['message_final'].iloc[index] = row['message_split_en']
<ipython-input-14-437a3419d628>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['en_original'].iloc[index] = 1
<ipython-input-14-437a3419d628>:13: SettingWithCopyW

In [ ]:
df_clean

,id,date,message,views,forwards,reactions,mime_type,👍,🙏,❤,😢,Total_emoijs,message_both,message_split_en,message_split_uk,message_lan,message_score,en_original,uk_trans,message_final
0,7045,2023-07-19 09:29:04+00:00,На ранковій селекторній нараді заслухав інформ...,21820.0,92.0,"[(❤, 515), (👍, 139), (🙏, 71), (😢, 20)]",text,139,71,515,20,745,0,None,None,uk,0.999997,0,1,None
1,7044,2023-07-18 20:55:39+00:00,I held a meeting of the Staff. The main issues...,107222.0,36.0,"[(❤, 3146), (👍, 340), (🙏, 206), (😢, 44)]",video/mp4,340,206,3146,44,3736,0,None,None,en,0.999997,0,0,None
2,7043,2023-07-18 19:41:43+00:00,Провів засідання Ставки. Основні питання – це ...,1371997.0,565.0,"[(❤, 3702), (👍, 370), (🙏, 210), (😢, 51)]",video/mp4,370,210,3702,51,4333,0,None,None,uk,0.999997,0,1,None
3,7042,2023-07-18 17:31:58+00:00,Зустрівся з президентом Японського агентства м...,327240.0,229.0,"[(❤, 3879), (👍, 480), (🙏, 192), (😢, 34)]",video/mp4,480,192,3879,34,4585,0,None,None,uk,0.999997,0,1,None
4,7041,2023-07-18 15:42:48+00:00,Війна – це зло. \nВійна – це трагедія.\nВійна ...,233477.0,294.0,"[(❤, 3689), (👍, 489), (🙏, 275), (😢, 59)]",video/mp4,489,275,3689,59,4512,1,\n\nWar is evil. \nWar is tragedy.\nWar is a t...,Війна – це зло. \nВійна – це трагедія.\nВійна ...,None,None,1,0,\n\nWar is evil. \nWar is tragedy.\nWar is a t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2979,729,2022-02-24 22:35:02+00:00,Сьогодні Росія атакувала всю територію України...,3486360.0,12478.0,"[(❤, 55), (😢, 5), (🙏, 4), (👍, 3)]",video/mp4,3,4,55,5,67,0,None,None,uk,0.999998,0,1,None
2980,728,2022-02-24 21:26:30+00:00,​​Закликав лідерів ЄС – учасників надзвичайног...,2103339.0,3286.0,"[(❤, 46), (👍, 5), (😢, 2)]",image/jpeg,5,0,46,2,53,0,None,None,uk,0.999997,0,1,None
2981,727,2022-02-24 15:35:31+00:00,Не Україна обрала шлях війни. Але Україна проп...,2637317.0,10002.0,"[(❤, 87), (🙏, 7), (😢, 5), (👍, 2)]",video/mp4,2,7,87,5,101,0,None,None,uk,0.999996,0,1,None
2982,726,2022-02-24 06:48:58+00:00,Я буду щогодини повідомляти вам актуальну і до...,3302767.0,19763.0,"[(❤, 129), (😢, 20), (👍, 1)]",video/mp4,1,0,129,20,150,0,None,None,uk,1.0,0,1,None


In [ ]:
print('Total mesages:', df_clean.shape[0])
print('Messages left over:', ((df_clean['en_original'] == 1) | (df_clean['uk_trans'] == 1)).sum())
print('Messages to drop:', ((df_clean['en_original'] == 0) & (df_clean['uk_trans'] == 0)).sum())
print('Of which pure English messages', sum(df_clean['message_lan'] == 'en'))
print('Of which messages in other languages than uk or en', ((df_clean['message_lan'] != 'uk') & (df_clean['message_lan'] != 'en') & (df_clean['message_both'] == 0)).sum())
print(' ')
print('Needs translations ', sum(df_clean['uk_trans'] == 1))
print('Original English messages ', sum(df_clean['en_original'] == 1))
print(' ')
print('UK messages ', sum(df_clean['message_lan'] == 'uk'))
print('Messages to translate from Ukraine to English', ((df_clean['message_lan'] == 'uk') & (df_clean['uk_trans'] == 1)).sum())
print('Messages whith both lan but en missing ', ((df_clean['message_both'] == 1) & (df_clean['message_split_en'].isna())).sum())

# Drop messages that are eihter pure en or another lan
df_clean_out = df_clean[((df_clean['en_original'] == 1) | (df_clean['uk_trans'] == 1))]
print('Final rows: ', df_clean_out.shape[0])

Total mesages: 2984
Messages left over: 2351
Messages to drop: 633
Of which pure English messages 612
Of which messages in other languages than uk or en 21
 
Needs translations  1187
Original English messages  1164
 
UK messages  1166
Messages to translate from Ukraine to English 1166
Messages whith both lan but en missing  21
Final rows:  2351


In [ ]:
# Output data
df_clean_out.to_csv('/content/drive/My Drive/01_LSE/2_Courses/ST/Dissertation/3_Ukraine_emotive_rethoric/Data/Inter/df_zelen_clean.csv')

In [ ]:
df_clean_out

,id,date,message,views,forwards,reactions,mime_type,👍,🙏,❤,😢,Total_emoijs,message_both,message_split_en,message_split_uk,message_lan,message_score,en_original,uk_trans,message_final
0,7045,2023-07-19 09:29:04+00:00,На ранковій селекторній нараді заслухав інформ...,21820.0,92.0,"[(❤, 515), (👍, 139), (🙏, 71), (😢, 20)]",text,139,71,515,20,745,0,None,None,uk,0.999997,0,1,None
2,7043,2023-07-18 19:41:43+00:00,Провів засідання Ставки. Основні питання – це ...,1371997.0,565.0,"[(❤, 3702), (👍, 370), (🙏, 210), (😢, 51)]",video/mp4,370,210,3702,51,4333,0,None,None,uk,0.999997,0,1,None
3,7042,2023-07-18 17:31:58+00:00,Зустрівся з президентом Японського агентства м...,327240.0,229.0,"[(❤, 3879), (👍, 480), (🙏, 192), (😢, 34)]",video/mp4,480,192,3879,34,4585,0,None,None,uk,0.999997,0,1,None
4,7041,2023-07-18 15:42:48+00:00,Війна – це зло. \nВійна – це трагедія.\nВійна ...,233477.0,294.0,"[(❤, 3689), (👍, 489), (🙏, 275), (😢, 59)]",video/mp4,489,275,3689,59,4512,1,\n\nWar is evil. \nWar is tragedy.\nWar is a t...,Війна – це зло. \nВійна – це трагедія.\nВійна ...,None,None,1,0,\n\nWar is evil. \nWar is tragedy.\nWar is a t...
5,7040,2023-07-18 10:39:31+00:00,Питання номер один на сьогоднішній Ставці – мо...,442594.0,255.0,"[(👍, 3322), (❤, 1159), (🙏, 296), (😢, 37)]",text,3322,296,1159,37,4814,0,None,None,uk,0.999999,0,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2979,729,2022-02-24 22:35:02+00:00,Сьогодні Росія атакувала всю територію України...,3486360.0,12478.0,"[(❤, 55), (😢, 5), (🙏, 4), (👍, 3)]",video/mp4,3,4,55,5,67,0,None,None,uk,0.999998,0,1,None
2980,728,2022-02-24 21:26:30+00:00,​​Закликав лідерів ЄС – учасників надзвичайног...,2103339.0,3286.0,"[(❤, 46), (👍, 5), (😢, 2)]",image/jpeg,5,0,46,2,53,0,None,None,uk,0.999997,0,1,None
2981,727,2022-02-24 15:35:31+00:00,Не Україна обрала шлях війни. Але Україна проп...,2637317.0,10002.0,"[(❤, 87), (🙏, 7), (😢, 5), (👍, 2)]",video/mp4,2,7,87,5,101,0,None,None,uk,0.999996,0,1,None
2982,726,2022-02-24 06:48:58+00:00,Я буду щогодини повідомляти вам актуальну і до...,3302767.0,19763.0,"[(❤, 129), (😢, 20), (👍, 1)]",video/mp4,1,0,129,20,150,0,None,None,uk,1.0,0,1,None
